In [1]:
import re

# 角色关系图谱
## 人物说明
1. 总共5个角色：林晓芳、张敏(林晓芳的女儿，比张翠山大)、张翠山(林晓芳的儿子)、令狐冲(张敏的儿子，比张无忌大)、张无忌(我)
2. *相互间关系全集*：表哥|儿子|舅舅|侄子|孙子|弟弟|表弟|爸爸|外甥|女儿|外婆|姐姐|奶奶|外孙|妈妈|姑妈
## 人物关系设定
2. 5个角色无其他会引起他们之间的关系会指向5个角色以外的关系的人物，如张无忌的表哥有且只有令狐冲

## 关系问答
3. 句式：**{关系描述}**是**{某个人}**的谁
4. **{关系描述}**从“你”开始描述
5. **{某个人}**可以是“你”、“你爸爸”、“你奶奶”、“张翠山”等

In [2]:
# 人物 ID
starts = {
    "你": 0,
    "爸爸": 1,
    "姑妈": 2,
    "表哥": 3,
    "奶奶": 4,
    "张无忌": 0,
    "张翠山": 1,
    "张敏": 2,
    "令狐冲": 3,
    "林晓芳": 4,
}

In [3]:
# 角色关系图谱
relation = {
    0: {
        "名字": "张无忌",
        "关系": "我",
        "自己": 0,
        "爸爸": 1,
        "姑妈": 2,
        "表哥": 3,
        "奶奶": 4,
    },
    1: {
        "名字": "张翠山",
        "关系": "爸爸",
        "儿子": 0,
        "自己": 1,
        "姐姐": 2,
        "外甥": 3,
        "妈妈": 4,
    },
    2: {
        "名字": "张敏",
        "关系": "姑妈",
        "侄子": 0,
        "弟弟": 1,
        "自己": 2,
        "儿子": 3,
        "妈妈": 4,
    },
    3: {
        "名字": "令狐冲",
        "关系": "表哥",
        "表弟": 0,
        "舅舅": 1,
        "自己": 3,
        "妈妈": 2,
        "外婆": 4,
    },
    4: {
        "名字": "林晓芳",
        "关系": "奶奶",
        "孙子": 0,
        "儿子": 1,
        "女儿": 2,
        "外孙": 3,
        "自己": 4,
    }
}

In [4]:
# 人物相互关系
rel_pattern = re.compile("你|表哥|儿子|舅舅|侄子|孙子|弟弟|表弟|爸爸|外甥|女儿|外婆|姐姐|奶奶|外孙|妈妈|姑妈")
# 问题中的相对谁的关系，如：你的谁；你爸爸的谁；令狐冲的谁
whom_patten = re.compile("张敏|林晓芳|令狐冲|张无忌|张翠山|你(奶奶|姑妈|表哥|爸爸)?")

def answer(quest):
    desc = quest.split("是")[0]
    quit = quest.split("是")[1]
    whom = whom_patten.search(quit).group(0)
    if len(whom) > 1 and whom[0] == "你": 
        whom = whom[1:]
    whom = starts[whom]
    match = rel_pattern.search(desc)
    if not match:
        return
    start, end = match.span()
    who = starts[desc[start:end]]
    desc = desc[end:]
    state = relation[who]
    while True:
        match = rel_pattern.search(desc)
        if not match:
            break
        start, end = match.span()
        between = desc[start:end]
        who = state[between]
        desc = desc[end:]
        state = relation[who]    
    for role_relation, role_id in relation[whom].items():
        if role_id == who:
            return role_relation

In [5]:
quests = [
    "你爸爸的姐姐的儿子的外婆是你的谁",
    "你爸爸的姐姐的儿子的外婆的孙子是你的谁",
    "爸爸的妈妈的女儿的儿子是你奶奶的谁",
    "你爸爸的姐姐的儿子的外婆是张翠山的谁",
]

In [6]:
for quest in quests:
    print(quest)
    print(answer(quest))
    print()

你爸爸的姐姐的儿子的外婆是你的谁
奶奶

你爸爸的姐姐的儿子的外婆的孙子是你的谁
自己

爸爸的妈妈的女儿的儿子是你奶奶的谁
外孙

你爸爸的姐姐的儿子的外婆是张翠山的谁
妈妈

